<a href="https://colab.research.google.com/github/son50git/PKT_001_Sentiment_Keras/blob/master/PKT_001_Rework_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from __future__ import print_function
import numpy as np 
np.random.seed(1337)

In [39]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential, Model  # Sequential not required for this example
from keras.layers import Dense, Dropout, Activation, Flatten, MaxPooling2D, Input # Input required
from keras.layers import Conv2D

In [40]:
batch_size = 32
num_classes = 10
epochs = 5

In [41]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [42]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [43]:
model = Sequential()

# from keras.layers.Conv2D
# filters: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).
# kernel_size: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window. 
# Can be a single integer to specify the same value for all spatial dimensions.

inputs = Input(shape=(32, 32, 3))

x = Conv2D(16, (3, 3), padding='same')(inputs) 
x = Activation('relu')(x) 

x = Conv2D(8, (3, 3))(x) 
x = Activation('relu')(x) 
x = MaxPooling2D(pool_size=(2, 2))(x) 
x = Flatten()(x) 

x = Dense(num_classes)(x) 

output = Activation('softmax')(x)

In [44]:
model = Model([inputs], output)


In [45]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 30, 30, 8)         1160      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 30, 8)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 8)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1800)             

In [46]:
# initiate RMSprop optimizer
import tensorflow as tf

# opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6) # this original line doesn't work
opt = tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [47]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Epoch 1/5
1563/1563 [==============================] - 60s 38ms/step - loss: 1.9838 - accuracy: 0.3025 - val_loss: 1.7912 - val_accuracy: 0.3854
Epoch 2/5
1563/1563 [==============================] - 59s 38ms/step - loss: 1.7223 - accuracy: 0.4025 - val_loss: 1.6562 - val_accuracy: 0.4297
Epoch 3/5
1563/1563 [==============================] - 60s 38ms/step - loss: 1.6285 - accuracy: 0.4354 - val_loss: 1.6174 - val_accuracy: 0.4361
Epoch 4/5
1563/1563 [==============================] - 59s 38ms/step - loss: 1.5724 - accuracy: 0.4523 - val_loss: 1.5432 - val_accuracy: 0.4644
Epoch 5/5
1563/1563 [==============================] - 59s 38ms/step - loss: 1.5287 - accuracy: 0.4670 - val_loss: 1.5241 - val_accuracy: 0.4660


In [48]:
!pip install h5py 

In [49]:
import h5py  # to ensure we have this package installed 

from keras.callbacks import ModelCheckpoint

filepath="checkpoints/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_acc', 
                             verbose=1, 
                             mode='max')

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          callbacks=[checkpoint])


Epoch 1/5
1563/1563 [==============================] - ETA: 0s - loss: 1.4902 - accuracy: 0.4806

KeyError: ignored